In [1]:
import pandas as pd
import glob

In [2]:
all_files = glob.glob('./data_for_cleaning/*.csv')
all_files.sort()

In [3]:
all_files

[]

In [4]:
for file in all_files:
    day_frame = pd.read_csv(file, index_col=None, header=0, sep= ';', low_memory=False)
    day_name = file.split('/')[2]
    
    namesToList = day_frame['name'].tolist()
    namesList = list(set(namesToList)) #created list of bikes
    
    latitude = 0
    longitude = 0
    
    new_df = pd.DataFrame(columns= ['lat', 'lng', 'time_begin', 'time_end','uid', 'address', 'name', 'bike', 'bike_racks', 'bikes', 
                                'booked_bikes', 'free_racks', 'free_special_racks', 'maintenance', 'number', 
                                'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type', 'city',
                                'country_code', 'company', 'timezone', 'available_bikes'] ) #empty dataframe with the new column structure
    
    for name in namesList: #repeat for each bike/bike name
        single_bike_records = day_frame[day_frame['name']==name] 
        #separating dataframe that has all the columns as day_frame but only for current bike

        first_row_data = single_bike_records.iloc[0] #saving first row data
        latitude, longitude = first_row_data['lat'], first_row_data['lng']
        new_df = new_df.append({'lat' : latitude, 'lng' : longitude,
                                'time_begin' : first_row_data['timestamp'], 'time_end' : 0,
                                'address' : first_row_data['address'], 'bike' : first_row_data['bike'], 
                                'bike_racks' : first_row_data['bike_racks'], 'bikes' : first_row_data['bikes'], 
                                'booked_bikes' : first_row_data['booked_bikes'], 'free_racks' : first_row_data['free_racks'], 
                                'free_special_racks' : first_row_data['free_special_racks'],
                                'maintenance' : first_row_data['maintenance'], 'name' : name, 'number' : first_row_data['number'], 
                                'place_type' : first_row_data['place_type'], 'rack_locks' : first_row_data['rack_locks'], 
                                'special_racks' : first_row_data['special_racks'], 'spot' : first_row_data['spot'], 
                                'terminal_type' : first_row_data['terminal_type'],'uid' : first_row_data['uid'], 
                                'city' : first_row_data['city'], 
                                'country_code' : first_row_data['country_code'], 'company' : first_row_data['company'], 
                                'timezone' : first_row_data['timezone'],
                                'available_bikes' : first_row_data['available_bikes']}, ignore_index=True) 
                                #saving the first row for bike - first location
        

        for index, row in single_bike_records.iterrows(): #itteration trough rows- possible different locations of the bike
            if latitude == row['lat'] and longitude == row['lng']:
                new_df.iloc[-1, new_df.columns.get_loc('time_end')] = row['timestamp'] #setting a value for time_end 
                #updating the last_time that bike was on a current location as long as we dont find the change in lat and lng

            else: #if we find the change in location
                new_df = new_df.append({'uid' : row['uid'] , 'lat' : row['lat'], 'lng' : row['lng'],
                                'time_begin' : row['timestamp'], 'time_end' : row['timestamp'],
                                'address' : row['address'], 'bike' : row['bike'], 
                                'bike_racks' : row['bike_racks'], 'bikes' : row['bikes'], 
                                'booked_bikes' : row['booked_bikes'], 'free_racks' : row['free_racks'], 
                                'free_special_racks' : row['free_special_racks'], 
                                'maintenance' : row['maintenance'], 'name' : row['name'], 'number' : row['number'], 
                                'place_type' : row['place_type'], 'rack_locks' : row['rack_locks'], 
                                'special_racks' : row['special_racks'], 'spot' : row['spot'], 
                                'terminal_type' : row['terminal_type'], 'city' : row['city'], 
                                'country_code' : row['country_code'], 'company' : row['company'], 
                                'timezone' : row['timezone'],
                                'available_bikes' : row['available_bikes']}, ignore_index=True) 
                #saving the new row if there is a change in location
                latitude, longitude = row['lat'], row['lng'] #updating to new location, continue searching for changes
                
    out_folder = './out_data/'
    new_df.to_csv(out_folder + day_name, sep=';', index=False) #saves each day file, one by one, when cleaning is finished 